In [21]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from optimum.onnxruntime import ORTModelForSequenceClassification
import numpy as np
import evaluate
import torch

In [9]:
raw_datasets = load_dataset("glue","sst2")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

In [6]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 1821/1821 [00:00<00:00, 21151.09 examples/s]


In [ ]:
training_args = TrainingArguments("test-trainer-sst2") # where You save

In [33]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

/tmp/ipykernel_62261/2671635385.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

  0%|          | 0/25257 [00:00<?, ?it/s]

  0%|          | 9/25257 [00:19<13:59:07,  1.99s/it]

KeyboardInterrupt: 

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])

In [ ]:
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("glue", "sst2")
metric.compute(predictions=preds, references=predictions.label_ids)